In [1]:
import numpy as np
np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
ticker='ADBE'
sentimentVector =  pd.read_csv('SentimentNews/'+ticker+'.csv')
price= pd.read_csv('/home/andrea/Desktop/NLFF/DataSetIndexesLabeled/indexes'+ticker+'.csv')
#alignment of data
maxdata=max(sentimentVector['initTime'])
mindata=min(sentimentVector['initTime'])
price=price[price['Unnamed: 0']>=mindata]
price=price[price['Unnamed: 0']<=maxdata]
#remove the time from data
sentimentVector=sentimentVector.drop(['Unnamed: 0', 'initTime' ], axis=1)
#2
#label because of the market and append values without datatime
#Important to set lenth of trend in this to see if news affect from today to today+tendwindowtime
#this is theorical (looks at difference betwwen price before news and price after news)because for make it real i have to change price.iloc[i-1]['open'] to price.iloc[i]['open']
#because the trader can buy only on price.iloc[i]['open'] that is today
trendwindowtime=[1,4,12,18,28,36,50]
mean=[10,20,30]
for m in mean:
    print('000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')
    print('000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

    print('Working on MEAN:',m)
    sentimentVectort1=pd.rolling_mean(sentimentVector,m)
    sentimentVectort=sentimentVectort1.iloc[m:]
    pricet=price.iloc[m:]
    for t in trendwindowtime:
        #1
    #label because of the maket and append values without data
    #simo theroy past trend
        x=[]
        y=[]
        print('============================================================')
        print('============================================================')
        print('Working on window:',t)
        for i in range(t,len(pricet)):
            #before change from 2 to price-1
            y.append(np.sign(pricet.iloc[i]['close']-pricet.iloc[i-t]['close']))
            #after change from 1 to price
            x.append(sentimentVectort.iloc[i].values)
        x=np.array(x)
        y=np.array(y)

        #Split betwwen train-validation and test
        train=0.8
        nt=math.ceil(len(x)*train)
        x_tv=[]
        y_tv=[]
        x_test=[]
        y_test=[]
        x_tv=x[:nt]
        y_tv=y[:nt]
        x_test=x[nt:]
        y_test=y[nt:]
        #Fairly sampling the tv
        posindex=np.where( y_tv > 0 )
        negindex=np.where( y_tv < 0 )

        yindex=[]
        nindex=min(len(posindex[0]),len(negindex[0]))

        #for i in range(1,nindex):
        y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
        x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))
        #Fairly sampling the test 50% 50%
        posindex=np.where( y_test > 0 )
        negindex=np.where( y_test < 0 )

        yindex=[]
        nindex=min(len(posindex[0]),len(negindex[0]))

        #for i in range(1,nindex):
        y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
        x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))

        nfold=10
        kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


        cspace=np.logspace(-3,4,30)
        gspace=np.logspace(-6,4,30)
        bestsvm=None
        maxacc=0
        cvacc=0
        maxg=0
        maxc=0
        print('Model Selection...')
        #model selection
        for c in cspace:
            for g in gspace:
                cvacc=0
                #crossvalidation
                for train_index, test_index in kf.split(x_tvnew):
                    x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                    y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                    rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                    rbf_svm.fit(x_train,y_train)
                    accuracy=rbf_svm.score(x_val,y_val)
                    cvacc=cvacc+accuracy/nfold

                if(cvacc>maxacc):
                    maxacc=cvacc
                    maxg=g
                    maxc=c
                    bestsvm=rbf_svm

        print('ACCURACY on val:',maxacc)
        print('Gamma:',maxg)
        print('C:',maxc)
        print(bestsvm)
        rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
        rbf_svm.fit(x_tvnew,y_tvnew)
        testacc=rbf_svm.score(x_testnew,y_testnew)
        print('Accuracy on TEST set:',testacc)
        print('LOOK WHAT APPENS IN TIME.....')
        cvacc=0
        bestc=maxc
        bestgamma=maxg
        x_tvr=[]
        y_tvr=[]

        stepsize=50
        nstep=len(x_tv)//stepsize
        accvector=[]
        niter=10
        for i in range(1,nstep):
            x_tvr=x_tv[0:i*stepsize]
            y_tvr=y_tv[0:i*stepsize]
            cvacc=0
            print(len(x_tvr))
            rbf_svm=svm.SVC(kernel='rbf',C=bestc,gamma=bestgamma)
            rbf_svm.fit(x_tvr,y_tvr)
            accuracy=rbf_svm.score(x_testnew,y_testnew)  
            accvector.append(accuracy)
        #out of the loop i put the last case in with i'm using all the dataset
        rbf_svm=svm.SVC(kernel='rbf',C=bestc,gamma=bestgamma)
        rbf_svm.fit(x_tvnew,y_tvnew)
        accuracy=rbf_svm.score(x_testnew,y_testnew)  
        accvector.append(accuracy)
        #VETTORE FINALE
        print('Final accuracy vector along time')
        print(accvector)
        plt.plot(accvector)
        plt.title('model accuracy on test in time:')
        plt.ylabel('accuracy')
        plt.xlabel('size train')
        plt.show()


000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Working on MEAN: 10
Working on window: 1


/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=10,center=False).mean()


Model Selection...
ACCURACY on val: 0.5229653552554316
Gamma: 923.6708571873884
C: 22.122162910704503
SVC(C=22.122162910704503, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=923.6708571873884,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.515527950310559
LOOK WHAT APPENS IN TIME.....
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
Final accuracy vector along time
[0.4968944099378882, 0.468944099378882, 0.4813664596273292, 0.48757763975155277, 0.4906832298136646, 0.515527950310559, 0.5124223602484472, 0.4751552795031056, 0.4782608695652174, 0.4906832298136646, 0.5, 0.4906832298136646, 0.4751552795031056, 0.4782608695652174, 0.453416149068323, 0.4440993788819876, 0.4503105590062112, 0.4658385093167702, 0.4720496894409938, 0.45652173913043476, 0.45962732919254656, 0.45652173913043476, 0.46273

Working on window: 4
Model Selection...
ACCURACY on val: 0.7311638779527558
Gamma: 38.56620421163472
C: 621.0169418915616
SVC(C=621.0169418915616, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=38.56620421163472,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.49101796407185627
LOOK WHAT APPENS IN TIME.....
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
Final accuracy vector along time
[0.5808383233532934, 0.5538922155688623, 0.5359281437125748, 0.5029940119760479, 0.5179640718562875, 0.48502994011976047, 0.49101796407185627, 0.5149700598802395, 0.5209580838323353, 0.5179640718562875, 0.5359281437125748, 0.5119760479041916, 0.5389221556886228, 0.5179640718562875, 0.5239520958083832, 0.5239520958083832, 0.5329341317365269, 0.5508982035928144, 0.5508982035928144, 0.5329341317365269, 0.53592814

Working on window: 12
Model Selection...
ACCURACY on val: 0.8203027669129365
Gamma: 85.31678524172814
C: 67.23357536499334
SVC(C=67.23357536499334, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=85.31678524172814,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.5434782608695652
LOOK WHAT APPENS IN TIME.....
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
Final accuracy vector along time
[0.5093167701863354, 0.5093167701863354, 0.531055900621118, 0.5372670807453416, 0.5093167701863354, 0.546583850931677, 0.48757763975155277, 0.43788819875776397, 0.46273291925465837, 0.4720496894409938, 0.4751552795031056, 0.4658385093167702, 0.48757763975155277, 0.4720496894409938, 0.5124223602484472, 0.4813664596273292, 0.484472049689441, 0.48757763975155277, 0.5031055900621118, 0.4782608695652174, 0.49068322

Working on window: 18
Model Selection...
ACCURACY on val: 0.8181386743886744
Gamma: 188.73918221350996
C: 38.56620421163472
SVC(C=38.56620421163472, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=188.73918221350996,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.4651898734177215
LOOK WHAT APPENS IN TIME.....
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
Final accuracy vector along time
[0.5221518987341772, 0.4208860759493671, 0.41139240506329117, 0.439873417721519, 0.4177215189873418, 0.4430379746835443, 0.41139240506329117, 0.4177215189873418, 0.40822784810126583, 0.40189873417721517, 0.40822784810126583, 0.41455696202531644, 0.41139240506329117, 0.40189873417721517, 0.33860759493670883, 0.34177215189873417, 0.35443037974683544, 0.41139240506329117, 0.4588607594936709, 0.49050632911392406

Working on window: 28
Model Selection...
ACCURACY on val: 0.85403286034354
Gamma: 85.31678524172814
C: 38.56620421163472
SVC(C=38.56620421163472, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=85.31678524172814,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.6357615894039735
LOOK WHAT APPENS IN TIME.....
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
Final accuracy vector along time
[0.5066225165562914, 0.5099337748344371, 0.5132450331125827, 0.5066225165562914, 0.5132450331125827, 0.5198675496688742, 0.4503311258278146, 0.4768211920529801, 0.47019867549668876, 0.5264900662251656, 0.5596026490066225, 0.5364238410596026, 0.5463576158940397, 0.5529801324503312, 0.5728476821192053, 0.5331125827814569, 0.5695364238410596, 0.6225165562913907, 0.6192052980132451, 0.6490066225165563, 0.6423841059602649

Working on window: 36
Model Selection...
ACCURACY on val: 0.8703164434090468
Gamma: 17.43328822199991
C: 10000.0
SVC(C=10000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=17.43328822199991,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.48322147651006714
LOOK WHAT APPENS IN TIME.....
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
Final accuracy vector along time
[0.5738255033557047, 0.5134228187919463, 0.5704697986577181, 0.4798657718120805, 0.44966442953020136, 0.46308724832214765, 0.4597315436241611, 0.4597315436241611, 0.4597315436241611, 0.47651006711409394, 0.5033557046979866, 0.4966442953020134, 0.37919463087248323, 0.3422818791946309, 0.3624161073825503, 0.3691275167785235, 0.4161073825503356, 0.48322147651006714, 0.4563758389261745, 0.5033557046979866, 0.5369127516778524, 0.516778523

Working on window: 50
Model Selection...
ACCURACY on val: 0.8721886446886448
Gamma: 188.73918221350996
C: 7.278953843983154
SVC(C=7.278953843983154, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=188.73918221350996,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy on TEST set: 0.5374149659863946
LOOK WHAT APPENS IN TIME.....
50


ValueError: The number of classes has to be greater than one; got 1

In [ ]:
values=[0.4732142857142857, 0.39285714285714285, 0.4791666666666667, 0.5119047619047619, 0.5148809523809523, 0.5178571428571429, 0.5089285714285714, 0.5178571428571429, 0.5386904761904762, 0.5238095238095238, 0.49702380952380953, 0.49404761904761907, 0.49107142857142855, 0.5148809523809523, 0.5119047619047619, 0.5267857142857143, 0.5267857142857143, 0.5089285714285714, 0.5178571428571429, 0.5089285714285714, 0.49702380952380953, 0.5297619047619048, 0.5148809523809523, 0.49702380952380953, 0.47023809523809523, 0.5029761904761905, 0.4791666666666667]
plt.plot(values)
plt.title('model accuracy on test in time:')
plt.ylabel('accuracy')
plt.xlabel('size train')
plt.show()